# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
import sys
sys.path.append("../")
from unityagents import UnityEnvironment
from network.DQN import DQNAgent
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(file_name="..\environment\Banana_Windows_x86_64\Banana.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
total_scores = []
scores = []
batch_size = 64
mean = 0
count = 0
eps = 1.0
eps_end = 0.01
decay = 0.999
max_t = 1000
gamma = 0.99
alpha = 1e-4
tua = 1e-3
max_memory_size = 50000
train = True  # set False, you can load the trained weights from file and skip the training process.
path = "../result/banana/"

with tf.Session() as session:

    brain_agent = DQNAgent(session, state_size, action_size, max_memory_size, gamma, alpha, tua)
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    while mean < 13 and train:
        env_info = env.reset(train_mode=True)[brain_name]
        score = 0
        time_b = time.time()
        loss = 0
        for i in range(max_t):
            if np.random.random() > eps:
                action = np.argmax(brain_agent.choose_action(state), axis=1)
            else:
                action = np.random.choice(action_size)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            score += reward
            brain_agent.store(state, action, reward, next_state, [done])
            state = next_state
            if brain_agent.step % 4 == 0:
                loss += brain_agent.learn(batch_size)
            if done:
                break
        scores.append(score)
        total_scores.append(score)
        eps = max(eps * decay, eps_end)
        print("\rEpisode: {},\tCurr Score: {},\tAverage Score: {:.2f},\tLoss:{:.4},\tEPS:{:.4},\tTime: {:.4}".format(count, score, np.mean(scores), loss/250.0, eps, time.time()-time_b), end="")
        if count % 100 == 0 and count > 0:
            mean = np.mean(scores)
            print("\rEpisode: {}, \tAverage Score: {:.2f}".format(count, mean))
            scores.clear()
        count += 1
        
    if train:
        saver.save(session, path)
        fig = plt.figure()
        plt.plot(range(len(total_scores)), total_scores)
        plt.ylabel('Score')
        plt.xlabel('Episode #')
        plt.show()
    else:
        saver.restore(session, path)
        
    for _ in range(10):
        done = False
        env_info = env.reset(train_mode=False)[brain_name]
        score = 0
        state = env_info.vector_observations[0]
        while not done:

            action = brain_agent.choose_action(state)
            env_info = env.step(action)[brain_name]  # send the action to the environment
            next_state = env_info.vector_observations[0]  # get the next state
            reward = env_info.rewards[0]  # get the reward
            done = env_info.local_done[0]  # see if episode has finished
            score += reward
            state = next_state
        print("Score is ", score)

NameError: name 'tf' is not defined